# Wide & Deep Learning model 구현

    * 꿈이 많은 사람의 이야기 블로그 참조하여 Wide & Deep Learning model 구현해보기
      - https://lsjsj92.tistory.com/597

* 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, SpatialDropout1D, Activation, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ReLU, PReLU, LeakyReLU, ELU
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from tensorflow.keras.utils import plot_model
import os

In [2]:
COLUMNS = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status", 
    "occupation", "relationship", "race", "gender", "capital_gain", "capital_loss", 
    "hours_per_week", "native_country", "income_bracket"
]

CATEGORICAL_COLUMNS = [
    "workclass", "education", "marital_status", "occupation", "relationship", 
    "race", "gender", "native_country"
]

CONTINUOUS_COLUMNS = [
    "age", "education_num", "capital_gain", "capital_loss", "hours_per_week"
]

In [3]:
len(COLUMNS)

15

In [4]:
len(CATEGORICAL_COLUMNS)

8

In [5]:
len(CONTINUOUS_COLUMNS)

5

    * COLUMNS : 전체 컬럼 목록
    * CATEGORICAL_COLUMNS : 카테고리 데이터 목록
        * workclass : 직종??
        * education : 학력
        * marital_status : 결혼/이혼 여부
        * occupation : 직업
        * relationship : 가족관계
        * race : 인종
        * gender : 성별
        * native_country : 국적
        
    * CONTINUOUS_COLUMNS : 연속적인 (숫자) 데이터 목록
        * age : 나이
        * education_num : ?? 학력을 숫자로 환산??
        * capital_gain : 
        * capital_loss : 
        * hours_per_week : 주당 근로시간
        
    * 그외
        * "fnlwgt" : 뭔지 모르지만?? 둘다 해당 안되니 신경쓰지 말자
        * income_bracket : 소득 수준 - label이라 둘다 해당 x 인듯
        
    => 각 사람의 개인 정보를 가지고 소득 수준을 맞추는 모델인듯?

Attribute Information:

Listing of attributes:

>50K, <=50K.

* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [6]:
train_data = pd.read_csv(os.getenv("HOME")+'/Hacka_refer_data/adult.data', names=COLUMNS)

In [7]:
print(train_data.shape)
train_data.head()

(32561, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
train_data.isna()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
32557,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
32558,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
32559,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [9]:
train_data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
gender            0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income_bracket    0
dtype: int64

In [10]:
test_data = pd.read_csv(os.getenv("HOME")+'/Hacka_refer_data/adult.test', skiprows = 1, names = COLUMNS)

In [11]:
print(test_data.shape)
test_data.head()

(16281, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [12]:
test_data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
gender            0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income_bracket    0
dtype: int64

* train data와 test data 이어 붙이기

In [13]:
data = pd.concat([train_data, test_data])

In [14]:
print(data.shape)
data.head()

(48842, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


* 라벨 생성
    - 라벨 = income_bracket

In [15]:
data['label'] = data['income_bracket'].apply(lambda x: ">50K" in x).astype(int)
# income_bracket 값이 ">50/K" 이면 1, 아니면 0으로 label 값을 지정
# astype(int) : T/F 를 0,1로 변환하는 듯

In [16]:
data.iloc[10:15]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket,label
10,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K,1
11,30,State-gov,141297,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,>50K,1
12,23,Private,122272,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,<=50K,0
13,32,Private,205019,Assoc-acdm,12,Never-married,Sales,Not-in-family,Black,Male,0,0,50,United-States,<=50K,0
14,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?,>50K,1


In [17]:
data.drop('income_bracket', axis = 1, inplace = True)

In [18]:
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


* y : label로 정의

In [19]:
y = data['label'].values
data.drop('label', axis = 1, inplace=True)

In [20]:
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [21]:
type(y)

numpy.ndarray

In [22]:
y

array([0, 0, 0, ..., 0, 0, 1])

In [23]:
for c in CATEGORICAL_COLUMNS:
    le = LabelEncoder()
    data[c] = le.fit_transform(data[c])

* LabelEncoder()의 fit_transform 메소드 확인용 코드

In [24]:
# temp_list = ["asd", "bvc", "fgh", "zxc"]
# temp_list2 = ["hgf", "bvc", "cvb"]
# temp_list3 = ["asd", "ghj", "fgh", "zxc", "asda", "fer"]
# temp_list_fit = le.fit_transform(temp_list)
# temp_list_fit2 = le.fit_transform(temp_list2)
# temp_list_fit3 = le.fit_transform(temp_list3)
# print(temp_list_fit)
# print(temp_list_fit2)
# print(temp_list_fit3)

 * LabelEncoder()의 fit_transform 메소드
    - 라벨 갯수에 따라 0으로 시작하는 정수로 반환하는 듯
    - 숫자는 알파벳 순으로
    
    
 * 생각해 볼 문제점
     - 이 방식으로 라벨을 인코딩 할 경우, 라벨데이터의 실제 의미와 무관하게 거리가 멀고 좁은 관계가 형성됨 => 추후 인코딩 방식 변경 고려

In [25]:
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5


## data를 Train, Test set으로 구분

In [26]:
train_size = len(train_data)
print(train_size)

32561


In [27]:
x_train = data.iloc[:train_size]
y_train = y[:train_size]
x_test = data.iloc[train_size:]
y_test = y[train_size:]

In [28]:
print(len(x_train), len(y_train), len(x_test), len(y_test))

32561 32561 16281 16281


## 카테고리 값들과 연속값들을 뽑아냄

In [29]:
x_train_category = np.array(x_train[CATEGORICAL_COLUMNS])
x_test_category = np.array(x_test[CATEGORICAL_COLUMNS])
x_train_continue = np.array(x_train[CONTINUOUS_COLUMNS], dtype='float64')
x_test_continue = np.array(x_test[CONTINUOUS_COLUMNS], dtype='float64')

In [30]:
print(x_train_category[:1] )
print(x_test_category[:1])
print(x_train_continue[:1])
print(x_test_continue[:1])

[[ 7  9  4  1  1  4  1 39]]
[[ 4  1  4  7  3  2  1 39]]
[[  39.   13. 2174.    0.   40.]]
[[25.  7.  0.  0. 40.]]


In [31]:
x_train_continue

array([[3.9000e+01, 1.3000e+01, 2.1740e+03, 0.0000e+00, 4.0000e+01],
       [5.0000e+01, 1.3000e+01, 0.0000e+00, 0.0000e+00, 1.3000e+01],
       [3.8000e+01, 9.0000e+00, 0.0000e+00, 0.0000e+00, 4.0000e+01],
       ...,
       [5.8000e+01, 9.0000e+00, 0.0000e+00, 0.0000e+00, 4.0000e+01],
       [2.2000e+01, 9.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+01],
       [5.2000e+01, 9.0000e+00, 1.5024e+04, 0.0000e+00, 4.0000e+01]])

In [32]:
x_train_category.shape

(32561, 8)

In [33]:
x_train_continue.shape

(32561, 5)

## 정규화

In [34]:
scaler = StandardScaler()
x_train_continue = scaler.fit_transform(x_train_continue)
x_test_continue = scaler.transform(x_test_continue)

In [35]:
print(x_train_continue[:1])
print(x_test_continue[:1])

[[ 0.03067056  1.13473876  0.1484529  -0.21665953 -0.03542945]]
[[-0.99570562 -1.19745882 -0.14592048 -0.21665953 -0.03542945]]


In [36]:
x_train_continue[0]

array([ 0.03067056,  1.13473876,  0.1484529 , -0.21665953, -0.03542945])

In [37]:
print(x_train_continue[0].sum())
print(x_train_continue[1].sum())
print(x_train_continue[2].sum())
print(x_train_continue[3].sum())
print(x_train_continue[4].sum())

1.0617732423626682
-0.6128853877817235
-0.8607111081353549
-0.5384215433976509
-0.03903855941908634


 - 정규화 한다고 다 더해서 1이 되거나 하지는 않는듯
 - 그렇다고 개별 값이 0~1은 아님

## Polynomial 하게 바꿔줌 
### (비선형적인 설정으로 선형 회귀를 확장하는 방법. 즉 다항식 함수로 바꿔줌)
    - 카테고리 값을 Polynomial로 바꿔줌

* sklearn.preprocessing.PolynomialFeatures 메소드
    - degree : 다항식 차수
    - interaction_only
        - default는 False
        - ex) degree = 3일 때, interaction_only=false 이면
            - a^2, a^3, b^2, b^3, ab, a^2*b, ab^2 Feature가 추가되고,
        - interaction_only=True 이면
            - ab만 추가됨
        

In [38]:
poly = PolynomialFeatures(degree=2, interaction_only=True)

In [39]:
x_train_category

array([[ 7,  9,  4, ...,  4,  1, 39],
       [ 6,  9,  2, ...,  4,  1, 39],
       [ 4, 11,  0, ...,  4,  1, 39],
       ...,
       [ 4, 11,  6, ...,  4,  0, 39],
       [ 4, 11,  4, ...,  4,  1, 39],
       [ 5, 11,  2, ...,  4,  0, 39]])

In [40]:
x_train_category.shape

(32561, 8)

In [41]:
x_test_category.shape

(16281, 8)

In [42]:
x_train_category_poly = poly.fit_transform(x_train_category)
x_test_category_poly = poly.fit_transform(x_test_category)

In [43]:
print(x_train_category_poly[:1])
print(x_train_category_poly[:1].shape)
print(x_test_category_poly[:1])
print(x_test_category_poly[:1].shape)

[[  1.   7.   9.   4.   1.   1.   4.   1.  39.  63.  28.   7.   7.  28.
    7. 273.  36.   9.   9.  36.   9. 351.   4.   4.  16.   4. 156.   1.
    4.   1.  39.   4.   1.  39.   4. 156.  39.]]
(1, 37)
[[  1.   4.   1.   4.   7.   3.   2.   1.  39.   4.  16.  28.  12.   8.
    4. 156.   4.   7.   3.   2.   1.  39.  28.  12.   8.   4. 156.  21.
   14.   7. 273.   6.   3. 117.   2.  78.  39.]]
(1, 37)


=> feature가 8개 이므로, interaction은 7+6+5+4+3+2+1=28개 이고,
상수항 1을 추가하여 8+28+1=37

In [44]:
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5


In [45]:
len(data)

48842

In [46]:
data[CATEGORICAL_COLUMNS[0]]

0        7
1        6
2        4
3        4
4        4
        ..
16276    4
16277    0
16278    4
16279    4
16280    5
Name: workclass, Length: 48842, dtype: int64

In [47]:
data[CATEGORICAL_COLUMNS].head()

,workclass,education,marital_status,occupation,relationship,race,gender,native_country
0,7,9,4,1,1,4,1,39
1,6,9,2,4,0,4,1,39
2,4,11,0,6,1,4,1,39
3,4,1,2,6,0,2,1,39
4,4,9,2,10,5,2,0,5


 * np.unique : np.arr 내 중복 제거

In [48]:
def get_deep_model():
    
    category_inputs = []
    category_embeds = []
    
    # Categorical Data Embedding
    for i in range(len(CATEGORICAL_COLUMNS)):
        
        # input - embedding - flatten 순으로 layer 쌓기
        input_i = Input(shape=(1,), dtype='int32')
        
        dim = len(np.unique(data[CATEGORICAL_COLUMNS[i]]))
        # dim : data에서 카테고리별 요소가 몇 종류인지?
        # 예시 data[학력] = '초', '중', '고' => dim = 3
        
        embed_dim = int(np.ceil(dim ** 0.5))
        # embedding 차원을 0.5배 정도로 수행? (연산은 루트 올림인디?)
        # 왜 임베딩 차원을 이렇게 하는지 추후 검토
        
        # 값 확인
        print(f"{i}의 dim 값 : {dim}")
        print(f"{i}의 embed_dim 값 : {embed_dim}")
        
        
        embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
        # dim : 데이터가 몇 종류 있는지 = 임베딩 벡터를 몇 개 뽑아낼 것인지
        # embed_dim : 임베딩 처리 후 벡터의 차원 = 임베딩 벡터를 몇 차원 벡터로 뽑아 낼 것인지
        # input_length : 입력 데이터 길이
        
        flatten_i = Flatten()(embed_i)
        # category 값을 임베딩환 벡터들을 flatten
        # 서로 다른 값의 벡터 요소가 합쳐져도 괜찮은지??
        # 어차피 class에 대한 순서가 있으니 상관 없을듯
        
        category_inputs.append(input_i)
        category_embeds.append(flatten_i)
        
    # continuous 데이터 input
    continue_input = Input(shape=(len(CONTINUOUS_COLUMNS),))
    continue_dense = Dense(256, use_bias=False)(continue_input)
    # use_bias = False로 하는 이유는??
    
    # category와 continue를 합침
    concat_embeds = concatenate([continue_dense] + category_embeds)
    concat_embeds = Activation('relu')(concat_embeds)
    # Activation 효과 다시 공부
    # relu 말고 다른 것은 어떤지??
    bn_concat = BatchNormalization()(concat_embeds)
    # Batch Normalization 효과 다시 공부
    
    fc1 = Dense(512, use_bias=False)(bn_concat)
    relu1 = ReLU()(fc1)
    bn1 = BatchNormalization()(relu1)
    fc2 = Dense(256, use_bias=False)(bn1)
    relu2 = ReLU()(fc2)
    bn2 = BatchNormalization()(relu2)
    fc3 = Dense(128)(bn2)
    relu3 = ReLU()(fc3)
    
    return category_inputs, continue_input, relu3

In [49]:
x_train_category

array([[ 7,  9,  4, ...,  4,  1, 39],
       [ 6,  9,  2, ...,  4,  1, 39],
       [ 4, 11,  0, ...,  4,  1, 39],
       ...,
       [ 4, 11,  6, ...,  4,  0, 39],
       [ 4, 11,  4, ...,  4,  1, 39],
       [ 5, 11,  2, ...,  4,  0, 39]])

In [50]:
x_train_category_poly

array([[  1.,   7.,   9., ...,   4., 156.,  39.],
       [  1.,   6.,   9., ...,   4., 156.,  39.],
       [  1.,   4.,  11., ...,   4., 156.,  39.],
       ...,
       [  1.,   4.,  11., ...,   0., 156.,   0.],
       [  1.,   4.,  11., ...,   4., 156.,  39.],
       [  1.,   5.,  11., ...,   0., 156.,   0.]])

In [51]:
x_train_category_poly.shape

(32561, 37)

In [52]:
def get_wide_model():
    dim = x_train_category_poly.shape[1]
    return Input(shape=(dim,))

# x_train_category_poly : 카테고리 데이터를 숫자로 바꾸고, Poly Feature를 추가한 것
# Poly Feature : a, b, c Feature를 이용해서 ab, bc, ca Feature를 만든것
# 데이터의 shape 만 가져옴

    * input - embedding - flatten 순으로 layer 쌓기

In [53]:
category_inputs, continue_input, deep_model = get_deep_model()
wide_model = get_wide_model()

0의 dim 값 : 9
0의 embed_dim 값 : 3
1의 dim 값 : 16
1의 embed_dim 값 : 4
2의 dim 값 : 7
2의 embed_dim 값 : 3
3의 dim 값 : 15
3의 embed_dim 값 : 4
4의 dim 값 : 6
4의 embed_dim 값 : 3
5의 dim 값 : 5
5의 embed_dim 값 : 3
6의 dim 값 : 2
6의 embed_dim 값 : 2
7의 dim 값 : 42
7의 embed_dim 값 : 7


In [54]:
wide_model

<tf.Tensor 'input_10:0' shape=(None, 37) dtype=float32>

In [55]:
deep_model

<tf.Tensor 're_lu_2/Identity:0' shape=(None, 128) dtype=float32>

### Wide모델과 Deep model을 합치기

* wide model, deep model 합치기

In [56]:
out_layer = concatenate([deep_model, wide_model])

In [57]:
out_layer

<tf.Tensor 'concatenate_1/Identity:0' shape=(None, 165) dtype=float32>

* 입력 값들 shape 확인

In [58]:
continue_input

<tf.Tensor 'input_9:0' shape=(None, 5) dtype=float32>

In [59]:
category_inputs

[<tf.Tensor 'input_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_3:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_4:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_5:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_6:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_7:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_8:0' shape=(None, 1) dtype=int32>]

In [60]:
wide_model

<tf.Tensor 'input_10:0' shape=(None, 37) dtype=float32>

* 입력 값들 종합하기

In [61]:
inputs = [continue_input] + category_inputs + [wide_model]

In [62]:
inputs

[<tf.Tensor 'input_9:0' shape=(None, 5) dtype=float32>,
 <tf.Tensor 'input_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_3:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_4:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_5:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_6:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_7:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_8:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'input_10:0' shape=(None, 37) dtype=float32>]

* wide model, deep model 합친 것을 한 점으로 모으기

In [63]:
output = Dense(1, activation='sigmoid')(out_layer)
# 여기서는 왜 relu가 아니고 sigmoid를 썻는지???
# 마지막 출력 값 범위를 0~1로 출력하기 위함 일듯?

In [64]:
model = Model(inputs=inputs, outputs=output)

In [65]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [66]:
checkpoint = ModelCheckpoint(filepath='./data/wide-deep.h5', monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

### 입력 데이터

    * 위에서 정의한 리스트 변수 inputs에 맞추어
    * continue 데이터 => category 데이터 => poly data 순으로 입력 값을 넣어준다

 * 데이터 확인

* x_train_continue
    - 숫자형 데이터(나이, 학력 수치, gain, loss, 업무시간)를 float형으로 변환하고
    - StandardScaler()를 통해 정규화 한 것

In [67]:
x_train_continue

array([[ 0.03067056,  1.13473876,  0.1484529 , -0.21665953, -0.03542945],
       [ 0.83710898,  1.13473876, -0.14592048, -0.21665953, -2.22215312],
       [-0.04264203, -0.42005962, -0.14592048, -0.21665953, -0.03542945],
       ...,
       [ 1.42360965, -0.42005962, -0.14592048, -0.21665953, -0.03542945],
       [-1.21564337, -0.42005962, -0.14592048, -0.21665953, -1.65522476],
       [ 0.98373415, -0.42005962,  1.88842434, -0.21665953, -0.03542945]])

In [68]:
x_train_continue.shape

(32561, 5)

* x_train_category
    - 범주형 데이터(업종, 학력, 결혼 여부, 직업, 가족관계, 인종, 성별, 국적)을  
    - LabelEncoder()를 통해 숫자(정수)로 변경(인코딩)

* x_train_category_poly
    - 각 feature를 2개씩 쌍을 지어 내적한 결과로 Feature를 추가한 데이터

In [69]:
x_train_category

array([[ 7,  9,  4, ...,  4,  1, 39],
       [ 6,  9,  2, ...,  4,  1, 39],
       [ 4, 11,  0, ...,  4,  1, 39],
       ...,
       [ 4, 11,  6, ...,  4,  0, 39],
       [ 4, 11,  4, ...,  4,  1, 39],
       [ 5, 11,  2, ...,  4,  0, 39]])

In [70]:
x_train_category.shape

(32561, 8)

In [71]:
[x_train_category[:, i] for i in range(x_train_category.shape[1])]

[array([7, 6, 4, ..., 4, 4, 5]),
 array([ 9,  9, 11, ..., 11, 11, 11]),
 array([4, 2, 0, ..., 6, 4, 2]),
 array([1, 4, 6, ..., 1, 1, 4]),
 array([1, 0, 1, ..., 4, 3, 5]),
 array([4, 4, 4, ..., 4, 4, 4]),
 array([1, 1, 1, ..., 0, 1, 0]),
 array([39, 39, 39, ..., 39, 39, 39])]

In [72]:
[x_train_category_poly]

[array([[  1.,   7.,   9., ...,   4., 156.,  39.],
        [  1.,   6.,   9., ...,   4., 156.,  39.],
        [  1.,   4.,  11., ...,   4., 156.,  39.],
        ...,
        [  1.,   4.,  11., ...,   0., 156.,   0.],
        [  1.,   4.,  11., ...,   4., 156.,  39.],
        [  1.,   5.,  11., ...,   0., 156.,   0.]])]

In [73]:
x_train_category_poly.shape

(32561, 37)

In [74]:
input_data = [x_train_continue] + [x_train_category[:, i] for i in range(x_train_category.shape[1])] + [x_train_category_poly]

* 모델 파라미터 설정

In [75]:
epochs = 30
optimizer = 'adam'
batch_size = 128

 * y : 수입이 50k 초과인지 아닌지
 * 초과면 1, 아니면 0

In [76]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [77]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(input_data, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.15, callbacks=[checkpoint, early_stopping])

Epoch 1/30
199/217 [==========================>...] - ETA: 0s - loss: 3.5337 - accuracy: 0.7856
Epoch 00001: val_loss improved from inf to 2.90201, saving model to ./data/wide-deep.h5
217/217 [==============================] - 1s 3ms/step - loss: 3.3363 - accuracy: 0.7872 - val_loss: 2.9020 - val_accuracy: 0.6868
Epoch 2/30
208/217 [===========================>..] - ETA: 0s - loss: 0.7597 - accuracy: 0.8101
Epoch 00002: val_loss improved from 2.90201 to 0.98260, saving model to ./data/wide-deep.h5
217/217 [==============================] - 1s 3ms/step - loss: 0.7566 - accuracy: 0.8099 - val_loss: 0.9826 - val_accuracy: 0.7365
Epoch 3/30
213/217 [============================>.] - ETA: 0s - loss: 0.5417 - accuracy: 0.8160
Epoch 00003: val_loss improved from 0.98260 to 0.51952, saving model to ./data/wide-deep.h5
217/217 [==============================] - 1s 3ms/step - loss: 0.5402 - accuracy: 0.8158 - val_loss: 0.5195 - val_accuracy: 0.8053
Epoch 4/30
211/217 [===========================

## 평가

In [78]:
# train input data와 같은 방식으로 test data를 input 형식에 맞추어줌
eval_input_data = [x_test_continue] + [x_test_category[:, i] for i in range(x_test_category.shape[1])] + [x_test_category_poly]

In [79]:
loss, acc = model.evaluate(eval_input_data, y_test)

509/509 [==============================] - 1s 1ms/step - loss: 0.3736 - accuracy: 0.8390


In [80]:
print(f'test_loss: {loss} - test_acc: {acc}')
# 문자열 앞에 f는 formating이었나? 확인

test_loss: 0.3735959827899933 - test_acc: 0.8390148282051086


* 모델 그래프로 그리기
    - 밑에 함수 수행하기 전에 터미널 명령어로 설치해야함
    1. pip install pydot
    2. sudo apt install graphviz

In [90]:
import pydot

In [91]:
plot_model(model, to_file=os.getenv("HOME")+'/Hacka_refer_model/model.png', show_shapes=True, show_layer_names=False)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [84]:
# 정규화된 데이터간 거리 구해보기
from scipy.spatial import distance

a = [0,0,0]
b = [0,3,4]
dist_ab = distance.euclidean(a,b)
print(dist_ab)

5.0
